In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV
import keras
import numpy as np
from keras.optimizers import SGD, Adam
from scikeras.wrappers import KerasRegressor
import pickle

2.1  Pobieranie danych

In [ ]:
housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test=train_test_split(housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid=train_test_split(X_train_full,y_train_full, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

2.2  Przeszukiwanie przestrzeni hiperparametrów przy pomocy scikit-learn

In [ ]:
param_distribs={"model__n_hidden": [0, 1, 2, 3],"model__n_neurons": np.arange(1, 100).tolist(),"model__learning_rate": reciprocal(3e-4,3e-2).rvs(1000).tolist(),"model__optimizer": ["adam", "sgd", "nesterov", "momentum"], "momentum": [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]}

def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, optimizer="adam", input_shape = [8], momentum = 0.0):
    model = keras.models.Sequential()

    model.add(keras.layers.InputLayer(input_shape=[8]))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(1))

    if optimizer == "sgd":
        opt = SGD(learning_rate=learning_rate)
    elif optimizer == "nesterov":
        opt = SGD(learning_rate=learning_rate, momentum=0.9, nesterov=True)
    elif optimizer == "adam":
        opt = Adam(learning_rate=learning_rate)
    elif optimizer == "momentum":
        opt = SGD(learning_rate=learning_rate, momentum=momentum)

    model.compile(loss="mse", optimizer=opt)
    return model

es=keras.callbacks.EarlyStopping(patience=10, min_delta=1.0, verbose=1)
keras_reg = KerasRegressor(build_model, callbacks=[es])

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3, verbose=2)
rnd_search_cv.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid))

best_params = rnd_search_cv.best_params_
print(best_params)
with open('rnd_search_params.pkl', 'wb') as f:
    pickle.dump(best_params, f)